In [ ]:
import pandas as pd
import re
import torch
from sentence_transformers import SentenceTransformer, util


Dataset cargado: 175352 filas
🔹 Cargando modelo SapBERT...


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext. Creating a new one with mean pooling.


🔹 Clasificando con SapBERT...


Batches:   0%|          | 0/2740 [00:00<?, ?it/s]


🔍 Reclasificando 15946 productos sin categoría...


Batches:   0%|          | 0/250 [00:00<?, ?it/s]


✅ Clasificación completada y guardada en '../output/dataset_clasificado_hibrido_mejorado.csv'


In [ ]:
df = pd.read_csv("../output/dataset_farmacia_integrado.csv")
df = df.dropna(subset=["nombre_normalizado"])
print(f"Dataset cargado: {df.shape[0]} filas")

In [ ]:
cat_names = [
    "Analgésicos / Antiinflamatorios",
    "Antibióticos",
    "Respiratorios / Antialérgicos",
    "Gastrointestinales",
    "Vitaminas y suplementos",
    "Ginecológicos / Urológicos",
    "Dermatológicos",
    "Material médico / Insumos",
    "Higiene y cosmética",
]
df["categoria_final"] = "Otros / No clasificado"


In [ ]:
print("🔹 Cargando modelo SapBERT...")
model = SentenceTransformer("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


In [ ]:
cat_embeds = model.encode(cat_names, convert_to_tensor=True, show_progress_bar=False).to(device)

In [ ]:
print("🔹 Clasificando con SapBERT...")
texts = df["nombre_normalizado"].astype(str).tolist()
embeddings = model.encode(texts, convert_to_tensor=True, show_progress_bar=True, batch_size=64).to(device)

sim_matrix = util.cos_sim(embeddings, cat_embeds).cpu()
best_cat_idx = sim_matrix.argmax(dim=1).tolist()
best_scores = sim_matrix.max(dim=1).values.tolist()

df["categoria_sapbert"] = [cat_names[i] for i in best_cat_idx]
df["confianza_sapbert"] = best_scores
reglas = {
    # Analgésicos / Antiinflamatorios
    r"analgesi|antiinflamatorio|antifebril|ibuprofeno|diclofenaco|naproxeno|ketorolaco|paracetamol|dolor|espasmo|miorelax|cefalea|artri|reuma|napron|neurobion|dolof|biolectra|antigripal forte": "Analgésicos / Antiinflamatorios",
    
    # Antibióticos
    r"amoxicilina|azitromicina|claritromicina|cefalexina|ampicilina|ciprofloxacina|clindamicina|levofloxacina|doxiciclina|eritromicina|trimetoprima|sulfametoxazol|penicilin|metronidazol|micina|infecc|bactrim|augmentin|amoxidal": "Antibióticos",
    
    # Respiratorios / Antialérgicos
    r"respir|tos|antigripal|expectorante|bronco|loratadina|cetirizina|antialerg|antihist|fexofenadina|gripal|nasal|aerosol|nebuliz|descongest|asma|mucolitic|pulm|alerg|garganta|vaporub|tabcin|next|resfrio|vick|fluimucil": "Respiratorios / Antialérgicos",
    
    # Gastrointestinales
    r"omeprazol|pantoprazol|ranitidina|domperidona|buscapina|viadil|digest|laxante|gastr|probio|fibra|hepatic|nause|vomit|diarrea|estreni|colon|bismut|antiacido|lactulosa|ulc|hepat|meb|omepr|gastro": "Gastrointestinales",
    
    # Vitaminas y suplementos
    r"vitamina|vitaminado|magnesio|hierro|calcio|colageno|suplement|omega|multivit|mineral|amino|ginseng|proteina|ensure|pediasure|fortisan|vital|tonico|jarabe|jalea|glucosamina|zinc|coenzima|gelatina|nutricion|bcaa|probio|energy|electrol|recovery|cereal": "Vitaminas y suplementos",
    
    # Ginecológicos / Urológicos
    r"embarazo|anticoncept|ovulo|estr|progester|fluconazol|metronidazol|gyn|ginecol|urolog|cistitis|urinari|prostata|sexual|preservativo|condon|lubric|candida|micosis|test|ovul|menstru|intimo|vagisil|fem|ovarial|clotrimazol|masculan|pantera|preservativo|condón": "Ginecológicos / Urológicos",
    
    # Dermatológicos
    r"pomada|crema|dermatol|acn|unguent|bepanthen|betametasona|hidrocortisona|talco|vaselin|bloqueador|locion|gel|solar|cicatriz|capilar|piel|fotoprotector|herida|hong|limpiador|corporal|manos|pies|antiarrug|facial|serum|mascarilla|eczema|fungicida": "Dermatológicos",
    
    # Material médico / Insumos
    r"jeringa|algodon|alcohol|termometro|guante|gasa|venda|hisopo|tensiometro|nebulizador|medidor|mascarilla|curita|kit|auxilio|sonda|ortoped|muleta|silla|rueda|aparato|glucosa|termal|tijera|baston|aposito|yeso|rodillera|tape|bandita|equipo|insumo|aparato|colector ": "Material médico / Insumos",
    
    # Higiene y cosmética
    r"shampoo|jabon|desodor|crema facial|locion corporal|gel de baño|cepillo|toalla|pañal|higien|cosmetic|bebe|perfume|boca|dental|afeit|depil|lente|esmalte|maquill|colonia|enjuague|infantil|toallita|bucal|labial|tinte|belleza|personal|fragancia|peine|cepill|colgate|listerine|rexona|nivea|dove": "Higiene y cosmética",
}

def aplicar_reglas(nombre):
    nombre = nombre.lower()
    for patron, categoria in reglas.items():
        if re.search(patron, nombre):
            return categoria
    return None

df["categoria_diccionario"] = df["nombre_normalizado"].apply(aplicar_reglas)


In [ ]:
UMBRAL_CONFIANZA = 0.50
df["categoria_final"] = df.apply(
    lambda x: x["categoria_diccionario"]
    if x["categoria_diccionario"] is not None
    else (x["categoria_sapbert"] if x["confianza_sapbert"] >= UMBRAL_CONFIANZA else "Otros / No clasificado"),
    axis=1,
)


In [ ]:
otros_idx = df[df["categoria_final"] == "Otros / No clasificado"].index
if len(otros_idx) > 0:
    print(f"\n🔍 Reclasificando {len(otros_idx)} productos sin categoría...")
    otros_embeds = model.encode(
        df.loc[otros_idx, "nombre_normalizado"].tolist(),
        convert_to_tensor=True,
        show_progress_bar=True,
        batch_size=64,
    ).to(device)
    sim_matrix_otros = util.cos_sim(otros_embeds, cat_embeds).cpu()
    best_cat_idx_otros = sim_matrix_otros.argmax(dim=1).tolist()
    df.loc[otros_idx, "categoria_final"] = [cat_names[i] for i in best_cat_idx_otros]


In [ ]:
df.to_csv("../output/dataset_clasificado_hibrido_mejorado.csv", index=False)
print("\n✅ Clasificación completada y guardada en '../output/dataset_clasificado_hibrido_mejorado.csv'")